# Cellular automaton models for metallurgy in MSE

*Authors: Enze Chen (University of California, Berkeley)*

![Cellular automata](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/CA_MSE.png)

This notebook teaches you the fundamentals of cellular automata (CA, plural) and some of their applications to MSE. I created this notebook because I find CA to be quite fascinating and capable of modeling complex physical phenomena, but they are often glossed over or ignored entirely in computational materials science courses. This is a shame because they're actually quite easy to understand! I hope you find this notebook instructive and enjoyable.

## How to run this notebook

If you are viewing this notebook on Google Colaboratory, then all the software is already set up for you (hooray). If you want to run the notebook locally, make sure all the Python libraries in the [`requirements.txt`](https://github.com/enze-chen/learning_modules/blob/master/requirements.txt) file are installed.

For pedagogical reasons, there are a few sections for you to complete the code in order to construct the CA. This is delineated with the dashed lines as follows, and you should **only change what's inside**. You don't have to edit the text or code anywhere else. I've also included "**TODO**" to separate the background context from the actual instructions.
```python
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #
```
If you edit the code in a cell, just press `Shift+Enter` to run it again. You have to execute **all** the code cells in this notebook from top to bottom (so don't skip around). A number `[#]` will appear to the left of the code cell once it's done executing. When done successfully, you'll be able to generate some nice animations that simulate real MSE phenomena!


## Acknowledgements

Most of these ideas were taken from Chapter 11 of [LeSar, R. *Introduction to Computational Materials Science*, Cambridge, 2013](https://www.cambridge.org/core/books/introduction-to-computational-materials-science/327CCEC340E5C466CE08D6A6FD8520E1). Another good source is the review paper by [Raabe, D. *Annual Review of Materials Research*, **32**, 2002](https://www.annualreviews.org/doi/10.1146/annurev.matsci.32.090601.152855). I thank my advisor [Prof. Mark Asta](https://mse.berkeley.edu/people_new/asta/) for encouraging me in my education-related pursuits. An interactive version of this notebook can be found online at [Google Colaboratory](https://colab.research.google.com/github/enze-chen/learning_modules/blob/master/mse/Cellular_automaton_models.ipynb). 

## Background

CA were discovered in the 1940s by [Stanislaw Ulam](https://en.wikipedia.org/wiki/Stanislaw_Ulam) and [John von Neumann](https://en.wikipedia.org/wiki/John_von_Neumann) at Los Alamos National Laboratory in their work on self-replicating models for artifical life. CA were then brought to the attention of the general public in the 1970s by [John Conway](https://en.wikipedia.org/wiki/John_Horton_Conway) in his namesake [Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) and in the 1980s by [Stephen Wolfram](https://en.wikipedia.org/wiki/Stephen_Wolfram), who in 2002 published his findings in his book [*A New Kind of Science*](https://en.wikipedia.org/wiki/A_New_Kind_of_Science). 

In one sentence, traditional CA model the **dynamic** evolution of a **discrete** system by applying a set of **deterministic** rules. We elaborate more on this in the following subsection. Despite their simplicity, CA can model a variety of complex biological (e.g. reproduction) and physical phenomena (e.g. non-equilibrium thermodynamics like the [Belousov-Zhabotinsky reaction](https://en.wikipedia.org/wiki/Belousov%E2%80%93Zhabotinsky_reaction)). Some CA can even emulate a universal [Turing machine](https://en.wikipedia.org/wiki/Turing_machine).

### Structure and rules

As its name implies, CA are composed of **cells** arranged in a discretized grid that is often rectangular, which we adopt here. The rectangular grid makes it convenient to index the cells with $(i,j)$, as shown in the figure below. In the traditional formulation, we restrict each cell to be in one of only two discrete **states** which you can think of as `on/off`, `alive/dead`, `dark/light`, or simply `1/0`.

![CA neighbors](https://raw.githubusercontent.com/enze-chen/learning_modules/master/fig/CA_neighbors.png)

The states of each cell will evolve over several discrete time steps (creating successive **generations**) according to a set of **transition rules** and the states of the other cells in their local **neighborhood**. The two common neighborhoods considered in CA simulations are the **von-Neumann neighborhood** (four nearest neighbors, orange above) and the **Moore neighborhood** (also include four next-nearest neighbors, orange and blue above). Cells along the boundary of the simulation domain will have a neighborhood specified by periodic boundary conditions (i.e. wraps to the other side). Note that all cells update their states **simultaneously**.

As an example of what these transition rules look like, here are the four rules that govern [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life), which considers the Moore neighborhood. Normally the user specifies their choice of the inital conditions of states.

1. Any live cell with fewer than two live neighbors dies, as if by underpopulation.
1. Any live cell with two or three live neighbors lives on to the next generation.
1. Any live cell with more than three live neighbors dies, as if by overpopulation.
1. Any dead cell with exactly three live neighbors becomes a live cell, as if by reproduction.

Seems simple right? It is! But these simple rules can lead to very complex behavior. Check out the Wikipedia page linked above for examples or [this insane YouTube video](https://www.youtube.com/watch?v=C2vgICfQawE).

## Our task

We will conduct two simulations using cellular automata: [recrystallization](https://en.wikipedia.org/wiki/Recrystallization_(metallurgy)) and [spinodal decomposition](https://en.wikipedia.org/wiki/Spinodal_decomposition). Recrystallization is the process by which strain-free and equiaxed grains nucleate and grow in a cold-worked metal, restoring its structure and properties to their original form. Spinodal decomposition is the spontaneous transformation of an alloy into two phases and notably 1) does not require nucleation, and 2) travels against the concentration gradient. These are two very important processes in MSE and we'll see if we can capture their qualitative behaviors.

## Import Python libraries

These are all the required Python libraries (so few!). We need [NumPy](https://numpy.org/) for the math and [Matplotlib](https://matplotlib.org/) for the plotting/animation. We'll use `scipy.stats.mode` to break ties.

**TODO**: Run this cell with `Shift+Enter` before starting the others.

In [ ]:
# Scientific computing libraries
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

# Interactivity libraries
import ipywidgets as widgets
from IPython.display import display

## Recrystallization

We will follow the implementation of [Hesselbarth, H.W. and Göbel, I.R. *Acta Metallurgica et Materialia*, **39**, 1991](http://www.sciencedirect.com/science/article/pii/0956715191901832), which had the following features:
* The geometry is a 2D square lattice.
* There are two states: recrystallized or not recrystallized
* Both von Neumann and Moore neighborhoods (and others) are considered.
* All sites are initialized to `0` (not recrystallized) and $N_{\text{embryo}}$ grain embryos were randomly assigned.

If we define the activity $A$ to be the **sum of recrystallized neighbors**, then the only rule is:

1. If $A \ge 1$ at time $t$, then the site would be recrystallized at time $t + 1$. Its identity would match that of the grains in the neighborhood, with ties broken arbitrarily.

In order to differentiate the grains, we'll assign each new grain to be a successive positive integer. We will also randomly select a few sites at random each time step and nucleate grain embryos at each of those sites that has not yet recrystallized.

**TODO**: Initialize the CA by creating a 2D NumPy array of zeros and randomly selecting a few sites to nucleate new grains. Some variables and hints that I suggest are:
* Create a variable `L` for the side length of the square grid. I suggest at least `50` for the final simulation, but start with `10` to test your code.
* Create a variable `n` for the number of new nucleation sites. I suggest `5`, but it's up to you.
* Create a variable `idn` to keep track of the different grains. We'll be using it later, but be smart about the initialization here.

In [ ]:
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #
cells = None   # placeholder for global variable

### Helper functions

To make our code more modular, we'll write a few helper functions to break down some of these tasks. To start with, let's write a function that initializes an $L \times L$ square grid and randomly chooses $n$ cells to nucleate new grain embryos.

**TODO**: Finish the `initialize()` function below, which is described by the docstring provided. Some hints are:

* Take advantage of the built-in NumPy functions [`np.zeros()`](https://numpy.org/doc/1.18/reference/generated/numpy.zeros.html), [`np.random.choice()`](https://numpy.org/devdocs/reference/random/generated/numpy.random.choice.html), and [`np.put()`](https://numpy.org/devdocs/reference/generated/numpy.put.html). See [this StackOverflow post](https://stackoverflow.com/a/33396710) for a hint on the last function, which makes changes **in-place** (nothing is returned).
* Remember, we would like each new grain to be a different integer value for the state so we can visually differentiate them.

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def initialize(L, n):
    """Initialize the CA with a few grain embryos.
    
    Args:
        N (int): Number of cells along one side of square grid.
        n (int): Number of new grain embryos to nucleate.
        
    Returns:
        A np.ndarray with initial grain embryos.
    """
    pass  # delete this and write your own code
# ---------------------- #
initialize(L=4, n=3)  # test

Next we will write a function that returns the states of the cells in the Moore neighborhood around $(i,j)$.

**TODO**: Finish the `neighbors()` function below, which is described by the docstring provided. Some hints are:
* You can store the neighbor states in a list first, and then convert it into a NumPy array at the end (we don't *need* a NumPy array but it makes the next part easier).
* When indexing into the neighbors, make sure to obey periodic boundary conditions (use the mod operator `%`).
* Make sure to exclude the center cell! You can just not add it to your neighbor list, or add everything and remove the center cell at the end.

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def neighbors(arr, i, j):
    """Get the states in the Moore neighborhood of a cell.
    
    Args:
        arr (np.ndarray): The grid of cells.
        i (int): The row index.
        j (int): The column index.
        
    Returns:
        A np.ndarray containing the states.
    """
    pass  # delete this and write your own code
# ---------------------- #
test = np.arange(16).reshape(4, 4)
print(test)
neighbors(arr=test, i=1, j=1)

The third helper function will perform the evolution at each time step. Remember that if the Moore neighborhood contains at least one grain, then the center cell will change to adopt the same grain. 

**TODO**: Finish the `evolve()` function below, which is described by the docstring provided. The evolution has two steps:

1. Grow the existing grains.
1. Nucleate new grain embryos.

Here are some hints:

* We include the global variable `idn` so we can properly number the new grains. You'll want to be incrementing this variable in the code you write.
* You only need to update a cell if it has not crystallized yet.
* Remember that CA update **simultaneously**. Therefore, the input array must be updated based on its initial values. This calls for a [copy](https://numpy.org/doc/1.18/reference/generated/numpy.ndarray.copy.html#numpy.ndarray.copy).
* If the neighbor list contains several different grains, you can decide how to want to break the tie. I think taking the mode using [`scipy.stats.mode()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html) is sensible and easy. Note that this method returns a tuple, and we only want the first element.
* For the selection of new nucleation sites, I recommend using `np.random.choice()` separately for the two indices. We'll choose the same number $n$ that you used initially.
* Remember that after a cell is chosen, we only nucleate a new grain embryo (with a new `idn`) if that cell has not yet recrystallized.

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def evolve(arr, n):
    """Evolve the CA to the next generation. First grow existing
    grains, and then try nucleating new embryos.
    
    Args:
        arr (np.ndarray): The grid of cells.
        n (int): The number of new cells to try nucleating
                 a new grain embryo in.
        
    Returns:
        A np.ndarray containing the updated states.
    """    
    # Grow existing grains

                    
    # Nucleate new embryos

    
    return arr
# ---------------------- #

### Interactive simulation

OK! If you completed the above steps correctly, the following widget should work. You'll see a button appear after running the following cell, and clicking that button will advance the time steps of recrystallization. When the grid is completely full, clicking the button again will reset to the initial state.

In [ ]:
button = widgets.Button(description="Evolve CA")
output = widgets.Output()
plt.rcParams.update({'figure.figsize':(7,7), 'image.cmap':'Spectral'})
display(button, output)

def on_button_clicked(b):
    """A function for the button widget.
    
    Args:
        b (Button): The referenced Button object.
        
    Returns:
        None, but a pyplot is displayed.
    """
    output.clear_output(wait=True)
    with output:
        if np.count_nonzero(cells==0) is 0:
            cells = initialize(L=L, n=n)
            idn = 5
        else:
            evolve(arr=cells, n=n)
        fig, ax = plt.subplots()
        ax.imshow(cells)
        ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
        plt.show()

button.on_click(on_button_clicked)

## Spinodal decomposition

With the help of the [Animation](https://matplotlib.org/3.2.1/api/animation_api.html) class in Matplotlib, we're going to make a movie showing the spinodal decomposition process! We will follow the implementation of [Oono, Y. and Puri, S. *Physical Review Letters*, **58**, 1987](https://link.aps.org/doi/10.1103/PhysRevLett.58.836), which had the following features:
* The geometry is a 2D rectangular lattice. 
* There is a tendency on a local level to phase segregate.
* Matter must be conserved, where local changes occur by diffusion.

Note that unlike traditional CA, each cell has a **continuous** state given by an **order parameter** defined as $\eta \equiv \Delta c = c_a - c_b$; that is, the concentration difference between the two components in our binary mixture. The physics of the system that are incorporated into the transition rules are:

1. The tendency to order, given by the **hyperbolic tangent** function $A \tanh \left( \eta_i(t) \right)$.
1. Diffusion of species, given by $-D_a \nabla^2 c_a = -D_a \left( \langle \langle \eta \rangle \rangle_i - \eta_i \right)$, which is the numerical approximation for the Laplacian. The term in double angle brackets is defined as

$$ \langle \langle \eta \rangle \rangle_i = \frac{1}{6} \sum_{j \in \text{nn}} \eta_j + \frac{1}{12} \sum_{j \in \text{snn}} \eta_j \tag{1}$$

where the first sum is over nearest neighbors and the second sum over second-nearest neighbors. Combining these two terms and enforcing conservation of mass gives the following transition rule:

$$ \eta_i(t+1) = \mathfrak{F} \left[ \eta_i \right] - \langle \langle \mathfrak{F} \left[ \eta_i \right] - \eta_i(t) \rangle \rangle,\ \quad \text{where} \quad \mathfrak{F} \left[ \eta_i \right] = A \tanh \left( \eta_i(t) \right) + D_a \left( \langle \langle \eta \rangle \rangle_i - \eta_i \right) \tag{2} $$


**TODO**: Start by initializing some constants for the simulation. Some hints are:

* You'll need to specify `A` and `D`. I recommend `1.5` and `0.7`, respectively.
* You'll need to specify the side length `N` of your square grid. I recommend `100`.

In [ ]:
# ---------------------- #
# YOUR CODE HERE

# ---------------------- #

We'll need a helper function to compute the averaging procedure described by the double angle brackets. Don't forget about periodic boundary conditions!

**TODO**: Finish the `avg_neighbors()` function, which is described by the docstring provided and given by Equation 1 above. Don't forget to write a suitable unit test!

In [ ]:
# ---------------------- #
# YOUR CODE HERE
def avg_neighbors(arr, i, j):
    """Compute the average value of the order parameter of the neighbors 
    at site (i,j) according to the method outlined by Oono and Puri.
    
    Args:
        arr (numpy.ndarray): The grid of cells.
        i (int): The row index.
        j (int): The column index.

    Returns:
        A float for the average order parameter of a site's Moore neighborhood.
    """
    pass  # delete this and write your own code
# ---------------------- #
# Write your own test here

Finally, to get the animation part to work, we have to define two functions, `init()` and `update()`. We will then use the [`FuncAnimation`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation) class to create an HTML movie from the sequence of images. I start with a few `frames` and long `interval`s for testing, but you'll want to change these to get a more illustrative movie. See the linked documentation for more details.

**TODO**: Finish the `update()` function below by implementing Equation 2 from above. As LeSar points out, "The calculation of the evolution of the conserved order parameter requires **two passes** through the lattice at each step: the change in each site is calculated... and then the two-dimensional average of that change is subtracted from its neighboring sites" (pg. 225). The emphasis is mine.

In [ ]:
plt.rcParams.update({'figure.figsize':(7,7), 'image.cmap':'Spectral', \
                     'animation.html':'jshtml'})
fig, ax = plt.subplots()
ax.tick_params(bottom=False, labelbottom=False, left=False, labelleft=False)
plt.close()

cells = np.random.default_rng().uniform(low=-0.1, high=0.1, size=(N, N))

def init():
    """Initialize the animation.
    
    Args:
        None
        
    Returns:
        A matplotlib.image.AxesImage object for the animation.
    """
    img = ax.imshow(cells)
    return (img,)

def update(dummy):
    """Update the frames in the animation.
    
    Args:
        dummy: Not used because animation doesn't depend on an 
               incrementing variable.
        
    Returns:
        A matplotlib.image.AxesImage object for the animation.
    """    
    # ---------------------- #
    # YOUR CODE HERE

    # ---------------------- #
    
    img = ax.imshow(cells)
    return (img,)

ani = animation.FuncAnimation(fig, update, init_func=init, frames=3, interval=500, repeat=False)
ani

## Conclusion

I hope this notebook taught you a little bit about cellular automata and metallurgy theory, and gave you some Python practice. It was only an introduction, and I recommend checking out the links in the [Introduction](#Acknowledgements) if you want to learn more about CA. If you have any remaining questions or ideas for this and other modules, please don't hesitate to reach out.



## Extensions

* Can you change the recrystallization CA simulation to consider a different neighborhood?
* Can you change the transition rules to be probabilistic instead of deterministic? For example, grain growth under impingement conditions can occur with a probability $p$ instead of deterministically.
* For spinodal decomposition, what happens if you change the values of $D$ and $A$? What about the initial range of $\eta$?

## Answers

If you found yourself stuck at certain points, I provide some sample answers [here](https://github.com/enze-chen/learning_modules/blob/master/data/answers.md#Cellular_automaton_models).